## Importing Dependencies 

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import time 

## Loading in Stock Symbols

In [2]:
# I got these csvs through another API, I didn't put it in here, as I only have limited uses (20 in total)
df = pd.read_csv("Stock_Code.csv")
sp500 = pd.read_csv("sp500.csv")

In [3]:
df.head()

,Code,Name,Country,Exchange,Currency,Type,Isin
0,A,Agilent Technologies Inc,USA,NYSE,USD,Common Stock,US00846U1016
1,AA,Alcoa Corp,USA,NYSE,USD,Common Stock,US0138721065
2,AAA,Listed Funds Trust - AAF First Priority CLO Bo...,USA,NYSE ARCA,USD,ETF,US53656F6566
3,AAC,AAC Holdings Inc,USA,NYSE,USD,Common Stock,KYG330321061
4,AAIC,Arlington Asset Investment Corp,USA,NYSE,USD,Common Stock,US0413562051


In [4]:
df.rename({"Code": "Symbol"}, axis = 1, inplace = True)

In [5]:
NYSE = df[(df["Exchange"] == "NYSE") & (df["Type"] == "Common Stock")]

In [6]:
sp500.head()

,Symbol,Name,Sector
0,MMM,3M,Industrials
1,AOS,A. O. Smith,Industrials
2,ABT,Abbott Laboratories,Health Care
3,ABBV,AbbVie,Health Care
4,ABMD,Abiomed,Health Care


In [7]:
sp500 = sp500[["Symbol", "Sector"]]

In [8]:
HealthCare = sp500[sp500["Sector"] == "Health Care"]

## Filter for Health Care Stocks 

-----

For these I focused on HealthCare Companies to narrow down are stock options (as there are too many). This is just tentative and can be changed.

In [9]:
combine = pd.merge(NYSE, HealthCare, on = "Symbol")

In [10]:
combine.head()

,Symbol,Name,Country,Exchange,Currency,Type,Isin,Sector
0,A,Agilent Technologies Inc,USA,NYSE,USD,Common Stock,US00846U1016,Health Care
1,ABBV,AbbVie Inc,USA,NYSE,USD,Common Stock,US00287Y1091,Health Care
2,ABC,AmerisourceBergen,USA,NYSE,USD,Common Stock,US03073E1055,Health Care
3,ABT,Abbott Laboratories,USA,NYSE,USD,Common Stock,US0028241000,Health Care
4,BAX,Baxter International Inc,USA,NYSE,USD,Common Stock,US0718131099,Health Care


## Extraction of  Meta Data to Rank Companies 
-----

* We can have a lot more columns than these, in fact over 50+ columns. I just choose a few - we should probably rank the companies that we would want to invest in based on key statistical insights. 

In [11]:
### Creation of Empty Columns to iterrow

combine["Book/Price"] = ""
combine["PEG Ratio"] = ""
combine["Price to Earning Ratio"] = ""
combine["Total Debt"] = ""
combine["Revenue"] = ""
combine["Total Debt/Equity"] = ""

for index, row in combine.iterrows():
    
    try:
    
        symbol = yf.Ticker(combine.loc[index, "Symbol"])
    
        print(f"Commence Extraction of {symbol} data")
        time.sleep(2)
        info = symbol.info
        combine.loc[index, "Book/Price"] = info["priceToBook"]
        combine.loc[index, "PEG Ratio"] = info["pegRatio"]
        combine.loc[index, "Price to Earning Ratio"] = info["forwardPE"]
        combine.loc[index, "Total Debt/Equity"] = info["debtToEquity"]
        combine.loc[index, "Revenue"] = info["totalRevenue"]
    
    except Exception:
        print(f"{symbol} not found")

Commence Extraction of yfinance.Ticker object <A> data
Commence Extraction of yfinance.Ticker object <ABBV> data
Commence Extraction of yfinance.Ticker object <ABC> data
Commence Extraction of yfinance.Ticker object <ABT> data
Commence Extraction of yfinance.Ticker object <BAX> data
Commence Extraction of yfinance.Ticker object <BDX> data
Commence Extraction of yfinance.Ticker object <BIO> data
Commence Extraction of yfinance.Ticker object <BMY> data
Commence Extraction of yfinance.Ticker object <BSX> data
Commence Extraction of yfinance.Ticker object <CAH> data
Commence Extraction of yfinance.Ticker object <CI> data
Commence Extraction of yfinance.Ticker object <CNC> data
Commence Extraction of yfinance.Ticker object <COO> data
Commence Extraction of yfinance.Ticker object <CRL> data
Commence Extraction of yfinance.Ticker object <CTLT> data
Commence Extraction of yfinance.Ticker object <CVS> data
Commence Extraction of yfinance.Ticker object <DGX> data
Commence Extraction of yfinance.

In [14]:
#combine.drop("Total Debt", axis = 1, inplace = True)

In [15]:
combine.head()

,Symbol,Name,Country,Exchange,Currency,Type,Isin,Sector,Book/Price,PEG Ratio,Price to Earning Ratio,Revenue,Total Debt/Equity
0,A,Agilent Technologies Inc,USA,NYSE,USD,Common Stock,US00846U1016,Health Care,8.633498,2.25,24.648087,6848000000,52.354
1,ABBV,AbbVie Inc,USA,NYSE,USD,Common Stock,US00287Y1091,Health Care,17.789696,-28.52,13.834049,57819000832,436.757
2,ABC,AmerisourceBergen,USA,NYSE,USD,Common Stock,US03073E1055,Health Care,None,1.68,13.520206,238587002880,9435.978
3,ABT,Abbott Laboratories,USA,NYSE,USD,Common Stock,US0028241000,Health Care,5.25683,2.47,24.390022,45029998592,46.204
4,BAX,Baxter International Inc,USA,NYSE,USD,Common Stock,US0718131099,Health Care,5.148894,2.77,15.532967,14739999744,302.671
